In [1]:
%reload_ext autoreload
%autoreload 2

In [4]:
import json
import polars as pl
from polars import col, lit
import altair as alt
import simrs
dir(simrs)

['History',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'bar',
 'run',
 'simrs']

In [6]:

ports = json.loads("""[{
      "id": "Rome",
      "market": {
        "table": {
          "Wheat": {
            "consumption": 30.0,
            "supply": 100.0,
            "production": 20.0,
            "pricer": {
              "base_supply": 0.0,
              "base_price": 100.0,
              "price_per_supply": -0.2
            }
          }
        }
      }
    },
    {
      "id": "Genoa",
      "market": {
        "table": {
          "Wheat": {
            "consumption": 20.0,
            "supply": 100.0,
            "production": 25.0,
            "pricer": {
              "base_supply": 0.0,
              "base_price": 100.0,
              "price_per_supply": -0.2
            }
          }
        }
      }
    }]"""
    )

inputFormat = {
  "opts": {"ticks": 1000},
  "edges": [["Rome", "Genoa"]],
  "agents": [],
  "ports": ports}

history = simrs.run(inputFormat)
tabular = history.tabular()

In [36]:
agents = pl.DataFrame(tabular["agents"])
markets = pl.DataFrame(tabular["markets"])
actions = pl.DataFrame(tabular["actions"])
markets.head()

consumption,good,port,price,pricer,production,supply,tick
f64,str,str,f64,struct[3],f64,f64,i64
20.0,"""Wheat""","""Genoa""",80.0,"{100.0,0.0,-0.2}",25.0,100.0,0
30.0,"""Wheat""","""Rome""",80.0,"{100.0,0.0,-0.2}",20.0,100.0,0
30.0,"""Wheat""","""Rome""",82.0,"{100.0,0.0,-0.2}",20.0,90.0,1
20.0,"""Wheat""","""Genoa""",79.0,"{100.0,0.0,-0.2}",25.0,105.0,1
20.0,"""Wheat""","""Genoa""",78.0,"{100.0,0.0,-0.2}",25.0,110.0,2


In [26]:
pricer = pl.DataFrame(tabular["markets"])
pricer.select(col("pricer").struct.field("base_price")).head()
pricer.select(col("pricer")).schema

{'pricer': Struct([Field('base_price', Float64), Field('base_supply', Float64), Field('price_per_supply', Float64)])}

In [30]:
markets.schema["pricer"]

Struct([Field('base_price', Float64), Field('base_supply', Float64), Field('price_per_supply', Float64)])

In [31]:
import hi

In [1]:
## Helpers to construct InputFormat
from typing import Tuple, Union


def route(from_, to):
    return [from_, to]

def agent(id, pos, cargo, coins, behavior):
    return {
        "id": id,
        "pos": pos,
        "cargo": cargo,
        "coins": coins,
        "behavior": behavior
    }

def market_info(consumption, supply, production, pricer):
    return {
        "consumption": consumption,
        "supply": supply,
        "production": production,
        "pricer": pricer
    }


def pricer(base_price=100, price_per_supply = -0.2, base_supply = 0):
    assert price_per_supply < 0
    return {
        "base_price": base_price,
        "base_supply": base_supply,
        "price_per_supply": price_per_supply
    }


def market(table: list[Tuple[str, dict]]):
    return {
        "table": {k: market_info(**v) for k, v in table},
    }
  
def port(id, market):
    return {
        "id": id,
        "market": market
    }


In [2]:
pricer()

{'base_price': 100, 'base_supply': 0, 'price_per_supply': -0.2}

Metrics for individual agent
- total coins
- coins per tick
- died?

Metrics for population of agents
- min, max, median, mean, std of agent coins

How well did the agents equalize prices?
- box plot of prices
- stddev of pricesf

Construct 'trades'
- bought Cargo at StartPort for BuyPrice
- sold Cargo at EndPort for SellPrice
- profit = SellPrice - BuyPrice
- Route = StartPort -> .. ->  EndPort
- RouteLength = len(Route)